In [2]:
import obi_one as obi

from pathlib import Path
import bluepysnap as snap

circuit_root = Path("../data/tiny_circuits")
circ_path = circuit_root / "N_10__top_nodes_dim6" / "circuit_config.json"
node_set = "S1nonbarrel_neurons"
node_id = 0

circ = snap.Circuit(circ_path)


/Users/james/Documents/obi/code/obi-one/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "name" in "Publication" shadows an attribute in parent "Block"
  warnings.warn(
/Users/james/Documents/obi/code/obi-one/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "name" in "CellMorphology" shadows an attribute in parent "Block"
  warnings.warn(
/Users/james/Documents/obi/code/obi-one/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "name" in "Subject" shadows an attribute in parent "Block"
  warnings.warn(


RuntimeError: Path `/Users/james/Documents/obi/additional_data/O1_data/circuit_config.json` is not a file

# Synapse selections

We demonstrate different ways of selecting subsets of afferent synapses on a neuron in a circuit.

The output of the selectors is always a pandas.DataFrame with one row per selected synapse. 

The DataFrame contains sufficient information in its columns to identify the synapses unequivocally. It specifies both the name of their edge population and the edge id within the population; this tuple is the identifier of an edge in Sonata. Additionally, the node population and node id of the presynaptic neuron are returned. "afferent_section_id", etc. are provided to specify the location of the synapse on the morphology, for example for visualizations.

Finally, some other, potentially useful statistics are also returned.

## Case 1: Random, unbiased selection

However, the second & third example show that section type, synapse class and node population can be filtered against.

Also note that both intrinsic and extrinsic synapses are considered! To avoid that, set "pre_node_population" to a list of non-virtual populations.

In [3]:
tst = obi.RandomlySelectedNumberOfSynapses(
    n=3
)
display(tst.synapses_on(circ, node_set, node_id))

tst = obi.RandomlySelectedNumberOfSynapses(
    n=3,
    section_types=(1,),
    pre_synapse_class="INH",
    pre_node_populations=("S1nonbarrel_neurons",)
)
display(tst.synapses_on(circ, node_set, node_id))

tst = obi.RandomlySelectedNumberOfSynapses(
    n=3,
    section_types=(3, 4),
    pre_node_populations=("POm",)
)
display(tst.synapses_on(circ, node_set, node_id))


,source_population,edge_population,edge_id,afferent_section_id,afferent_segment_id,afferent_segment_offset,@source_node,@target_node,afferent_section_type
45,POm,POm__S1nonbarrel_neurons__chemical,45,1615,105,0.483725,137,0,3
118,VPM,VPM__S1nonbarrel_neurons__chemical,67,1667,4,0.292370,112,0,3
16,POm,POm__S1nonbarrel_neurons__chemical,16,1651,94,0.504319,49,0,3


,source_population,edge_population,edge_id,afferent_section_id,afferent_segment_id,afferent_segment_offset,@source_node,@target_node,synapse_class,afferent_section_type


,source_population,edge_population,edge_id,afferent_section_id,afferent_segment_id,afferent_segment_offset,@source_node,@target_node,afferent_section_type
29,POm,POm__S1nonbarrel_neurons__chemical,29,1654,0,0.833073,83,0,3
11,POm,POm__S1nonbarrel_neurons__chemical,11,1655,0,6.718178,28,0,3
10,POm,POm__S1nonbarrel_neurons__chemical,10,1631,2,8.424549,25,0,3


### Note on insufficient synapses

In general, if there are fewer valid synapses than requested, no exception is raised. Instead simply the lower number of synapses is returned.
The code has a "raise_insufficient" flag that can change the behavior, but it currently is not exposed to the user.

Another question is: What happens if filter, e.g., by presynaptic synapse class, but the presynaptic nodes do not have a synapse_class attribute? This is set by the optional consider_nan_pass parameter.

In [4]:
# Requesting EXC synapses on the axon. There are none. This will return empty
tst = obi.RandomlySelectedNumberOfSynapses(
    n=3,
    section_types=(2, ),
    pre_synapse_class="EXC"
)
display(tst.synapses_on(circ, node_set, node_id))

# Default behavior: Even though the POm population has no synapse_class attribute, they will not be filtered. This returns 3 synapses
tst = obi.RandomlySelectedNumberOfSynapses(
    n=3,
    pre_synapse_class="EXC",
    pre_node_populations=("POm",)
)
display(tst.synapses_on(circ, node_set, node_id))

# More strict: POm, which have no synapse_class will be filtered. Hence, no synapses can be returned.
tst = obi.RandomlySelectedNumberOfSynapses(
    n=3,
    pre_synapse_class="EXC",
    pre_node_populations=("POm",),
    consider_nan_pass=False
)
display(tst.synapses_on(circ, node_set, node_id))

,source_population,edge_population,edge_id,afferent_section_id,afferent_segment_id,afferent_segment_offset,@source_node,@target_node,synapse_class,afferent_section_type


,source_population,edge_population,edge_id,afferent_section_id,afferent_segment_id,afferent_segment_offset,@source_node,@target_node,synapse_class,afferent_section_type
29,POm,POm__S1nonbarrel_neurons__chemical,29,1654,0,0.833073,83,0,NaN,3
11,POm,POm__S1nonbarrel_neurons__chemical,11,1655,0,6.718178,28,0,NaN,3
10,POm,POm__S1nonbarrel_neurons__chemical,10,1631,2,8.424549,25,0,NaN,3


,source_population,edge_population,edge_id,afferent_section_id,afferent_segment_id,afferent_segment_offset,@source_node,@target_node,synapse_class,afferent_section_type


## Case 2: Synapse clusters

There are two options: (1) Return all synapses that are within a specified maximum distance of the center of a cluster. (2) Return the n closest synapses to a cluster center.

Now, an additional column "cluster_id" is added.

In [5]:
# All within the distance
tst = obi.ClusteredSynapsesByMaxDistance(
    n_clusters=3,
    cluster_max_distance=25.0
)
display(tst.synapses_on(circ, node_set, node_id))

# The 25 closest
tst = obi.ClusteredSynapsesByCount(
    n_clusters=3,
    n_per_cluster=25,
    pre_node_populations=("VPM", "POm")  # Additional filters still enabled
)
display(tst.synapses_on(circ, node_set, node_id))


,cluster_id,source_population,edge_population,edge_id,afferent_section_id,afferent_segment_id,afferent_segment_offset,@source_node,@target_node,afferent_section_type
11,0,POm,POm__S1nonbarrel_neurons__chemical,11,1655,0,6.718178,28,0,3
19,0,POm,POm__S1nonbarrel_neurons__chemical,19,1654,0,5.291983,51,0,3
28,0,POm,POm__S1nonbarrel_neurons__chemical,28,1655,0,6.729255,71,0,3
29,0,POm,POm__S1nonbarrel_neurons__chemical,29,1654,0,0.833073,83,0,3
34,0,POm,POm__S1nonbarrel_neurons__chemical,34,1653,55,5.368964,99,0,3
47,0,POm,POm__S1nonbarrel_neurons__chemical,47,1653,55,5.758632,137,0,3
94,1,VPM,VPM__S1nonbarrel_neurons__chemical,43,1667,4,5.384574,74,0,3
102,1,VPM,VPM__S1nonbarrel_neurons__chemical,51,1667,1,3.009423,89,0,3
104,1,VPM,VPM__S1nonbarrel_neurons__chemical,53,1667,4,1.460957,93,0,3
118,1,VPM,VPM__S1nonbarrel_neurons__chemical,67,1667,4,0.292370,112,0,3


,cluster_id,source_population,edge_population,edge_id,afferent_section_id,afferent_segment_id,afferent_segment_offset,@source_node,@target_node,afferent_section_type
47,0,POm,POm__S1nonbarrel_neurons__chemical,47,1653,55,5.758632,137,0,3
34,0,POm,POm__S1nonbarrel_neurons__chemical,34,1653,55,5.368964,99,0,3
29,0,POm,POm__S1nonbarrel_neurons__chemical,29,1654,0,0.833073,83,0,3
19,0,POm,POm__S1nonbarrel_neurons__chemical,19,1654,0,5.291983,51,0,3
11,0,POm,POm__S1nonbarrel_neurons__chemical,11,1655,0,6.718178,28,0,3
...,...,...,...,...,...,...,...,...,...,...
126,2,VPM,VPM__S1nonbarrel_neurons__chemical,75,1683,70,0.791971,139,0,3
81,2,VPM,VPM__S1nonbarrel_neurons__chemical,30,1680,118,0.956375,59,0,3
68,2,VPM,VPM__S1nonbarrel_neurons__chemical,17,1688,39,5.709842,36,0,3
73,2,VPM,VPM__S1nonbarrel_neurons__chemical,22,1688,40,0.389723,53,0,3


## Case 3: Also consider path distance


In [6]:
# All within the distance
tst = obi.ClusteredPDSynapsesByMaxDistance(
    n_clusters=3,
    cluster_max_distance=25.0,
    soma_pd_mean=100.0,
    soma_pd_sd=10.0
)
display(tst.synapses_on(circ, node_set, node_id))

# The 25 closest
tst = obi.ClusteredPDSynapsesByCount(
    n_clusters=3,
    n_per_cluster=25,
    soma_pd_mean=400.0,
    soma_pd_sd=10.0,
    pre_node_populations=("VPM", "POm")
)
display(tst.synapses_on(circ, node_set, node_id))

,cluster_id,source_population,edge_population,edge_id,afferent_section_id,afferent_segment_id,afferent_segment_offset,@source_node,@target_node,afferent_section_type
90,0,VPM,VPM__S1nonbarrel_neurons__chemical,39,1695,27,0.082972,66,0,3
91,0,VPM,VPM__S1nonbarrel_neurons__chemical,40,1695,17,0.388120,71,0,3
114,0,VPM,VPM__S1nonbarrel_neurons__chemical,63,1695,11,0.006672,109,0,3
130,1,VPM,VPM__S1nonbarrel_neurons__chemical,79,1665,56,6.958955,157,0,3
132,1,VPM,VPM__S1nonbarrel_neurons__chemical,81,1665,56,4.684038,165,0,3
134,1,VPM,VPM__S1nonbarrel_neurons__chemical,83,1665,57,0.801131,178,0,3
111,2,VPM,VPM__S1nonbarrel_neurons__chemical,60,1683,46,0.310458,102,0,3
127,2,VPM,VPM__S1nonbarrel_neurons__chemical,76,1683,34,0.126600,156,0,3


,cluster_id,source_population,edge_population,edge_id,afferent_section_id,afferent_segment_id,afferent_segment_offset,@source_node,@target_node,afferent_section_type
19,0,POm,POm__S1nonbarrel_neurons__chemical,19,1654,0,5.291983,51,0,3
29,0,POm,POm__S1nonbarrel_neurons__chemical,29,1654,0,0.833073,83,0,3
47,0,POm,POm__S1nonbarrel_neurons__chemical,47,1653,55,5.758632,137,0,3
11,0,POm,POm__S1nonbarrel_neurons__chemical,11,1655,0,6.718178,28,0,3
28,0,POm,POm__S1nonbarrel_neurons__chemical,28,1655,0,6.729255,71,0,3
...,...,...,...,...,...,...,...,...,...,...
107,2,VPM,VPM__S1nonbarrel_neurons__chemical,56,1664,14,6.393769,97,0,3
30,2,POm,POm__S1nonbarrel_neurons__chemical,30,1615,160,0.487473,87,0,3
97,2,VPM,VPM__S1nonbarrel_neurons__chemical,46,1693,20,0.814003,83,0,3
136,2,VPM,VPM__S1nonbarrel_neurons__chemical,85,1677,69,0.036121,180,0,3


## Case 4: Consider connections instead of individual synapses

This one may be useful for spike replay. The idea is that instead of selecting individual synapses, the code selects from presynaptic neurons.

This can be applied by setting "merge_multiple_syns_con" to True in any of the Blocks. 

In that case, path distances are considered by averaging them over synapse from the same presynaptic neuron. That is, for each presynaptic neuron, its path distance to all others and to the soma is the mean of its synapses onto the morphology. 
It is up to the user to decide how much sense that makes in their use case. Otherwise, they can stick to the non-path-distance based selectors.

#### Note
The output in that case is much leaner: It contains only the required information to identify the presynaptic neuron selected. This is because for most of the outputs of the dataframe, such as section id, averaging does not make sense. So they are dropped instead.

In [7]:
tst = obi.PathDistanceConstrainedNumberOfSynapses(
    n=25,
    soma_pd_mean=400.0,
    soma_pd_sd=10.0,
    pre_node_populations=("VPM", "POm"),
    merge_multiple_syns_con=True
)
display(tst.synapses_on(circ, node_set, node_id))

,source_population,@source_node
48,VPM,83
27,POm,136
32,VPM,16
22,POm,101
31,VPM,12
45,VPM,71
30,VPM,8
60,VPM,139
55,VPM,112
62,VPM,157
